# 🏥 Salud y Farmacéutico: calidad, trazabilidad, acceso
KPIs: readmisión, tiempos de espera, abastecimiento, adherencia.

## Casos de uso
- Integración HL7/FHIR, interoperabilidad
- Optimización de camas y urgencias
- Farmacovigilancia y cadenas de frío
- Modelos de propensión a adherencia y abandono

In [1]:
# Cálculo simple de tiempos promedio en urgencias
import statistics as st
esperas = [15, 20, 45, 30, 25]
st.mean(esperas), st.median(esperas)

(27, 25)

## Arquitectura
- Conectores FHIR/HL7, limpieza y seudonimización
- Lakehouse con privacidad diferencial en research
- APIs para trazabilidad y dashboards clínicos

## 💼 Valor para el negocio en Salud
- Menos readmisiones y tiempos de espera → mejor calidad y costo por paciente.
- Abastecimiento óptimo de fármacos e insumos → menos quiebres y mermas.
- Cadena de frío y trazabilidad → cumplimiento y confianza.


In [2]:
# Práctica: tiempos de espera antes y después (toy)
import statistics as st
espera_base = [15, 20, 45, 30, 25, 35, 40]
espera_mejora = [12, 18, 32, 24, 22, 28, 30]
print('Promedio base:', round(st.mean(espera_base),1), 'min')
print('Promedio mejorado:', round(st.mean(espera_mejora),1), 'min')
print('Reducción %:', round(100*(st.mean(espera_base)-st.mean(espera_mejora))/st.mean(espera_base),1))


Promedio base: 30 min
Promedio mejorado: 23.7 min
Reducción %: 21.0


## 🏗️ Arquitectura y privacidad
- Ingesta HL7/FHIR con limpieza y seudonimización.
- Lakehouse con zonas de acceso y privacidad diferencial en research.
- APIs para trazabilidad (cadenas de frío) e interoperabilidad.


In [3]:
# Visual: comparación de esperas (toy)
import plotly.express as px
import pandas as pd
vis = pd.DataFrame({
    'grupo':['base']*len(espera_base)+['mejora']*len(espera_mejora),
    'minutos':espera_base+espera_mejora
})
fig = px.box(vis, x='grupo', y='minutos', title='Tiempos de espera: base vs mejora')
fig.show()


## ✅ Checklist de estrategia de negocio y datos
- Objetivo salud (calidad, acceso, costo por paciente): …
- KPIs: readmisión, espera, abastecimiento, adherencia.
- Palancas: interoperabilidad HL7/FHIR, trazabilidad, analítica operativa.
- Datos: clínicos/operativos; seudonimización y minimización PII.
- Gobierno: dominios, contratos, privacidad diferencial.
- SLOs: latencia tableros, disponibilidad, costo.
- ROI/supuestos: …
- Riesgos: regulatorios, seguridad, sesgos.


## 🔗 Puente Estrategia ↔ Ingeniería de Datos
- OKR: Tiempo de espera -20%, readmisión -10% → KPI negocio: espera, readmisión → KPI datos: interoperabilidad (FHIR), calidad y privacidad → Capacidades: ingesta HL7/FHIR + DQ + seudonimización → Decisión: asignación de recursos y protocolos → Impacto: costo por paciente↓.

```
[Registros clínicos]→[FHIR/HL7]→[Calidad+Privacidad]→[Métricas espera]→[Umbral]→[Asignación de recursos]→[Δ costo]
```


In [4]:
# contrato_de_datos (salud)
contrato_datos = {
    "objetivo_negocio": "Tiempo de espera -20%",
    "kpi_negocio": "Minutos de espera",
    "kpi_datos": {"interoperabilidad": "FHIR OK", "privacidad": "PII tokenizada"},
    "dataset": ["citas", "urgencias", "insumos"],
    "propietario": "Dominio Clínico",
    "slo": {"freshness_min": "<=15"},
    "decision": "Asignación de personal y camas",
    "impacto_economico_usd": 120000,
}
print(contrato_datos)


{'objetivo_negocio': 'Tiempo de espera -20%', 'kpi_negocio': 'Minutos de espera', 'kpi_datos': {'interoperabilidad': 'FHIR OK', 'privacidad': 'PII tokenizada'}, 'dataset': ['citas', 'urgencias', 'insumos'], 'propietario': 'Dominio Clínico', 'slo': {'freshness_min': '<=15'}, 'decision': 'Asignación de personal y camas', 'impacto_economico_usd': 120000}


## 📋 Caso de uso: Optimización de urgencias con datos interoperables

**Contexto de negocio:** Un hospital tiene tiempos de espera promedio de 45 min en urgencias, con picos de 2h. Objetivo: reducir a <30 min para mejorar NPS y evitar sanciones regulatorias.

**Problema sin ingeniería de datos:**
- Sistemas de citas, urgencias y camas desconectados.
- Registros clínicos en formatos propietarios (no FHIR).
- Planificación de personal basada en históricos de hace 6 meses.

**Solución con ingeniería de datos:**
1. Ingesta HL7/FHIR: normalización de registros de citas, urgencias, camas e insumos.
2. Pipeline de calidad: validación de PII, seudonimización automática.
3. Modelo de predicción de demanda: forecasting de llegadas por hora/día con features de estacionalidad.
4. Dashboard en tiempo real: ocupación de camas, personal disponible, pacientes en espera.
5. Privacidad diferencial en datasets de investigación.

**Resultado:**
- Tiempo de espera: 45 min → 28 min promedio.
- Ocupación de camas: visible en tiempo real vs. lag de 4h.
- Cumplimiento regulatorio: auditores pueden validar privacidad con linaje completo.
- Ahorro anual: $800k (mejor uso de recursos + menos sanciones).


In [5]:
# Ejercicio: pipeline de calidad con seudonimización
import pandas as pd
import hashlib

# Simular registros clínicos con PII
pacientes = pd.DataFrame({
    'dni': ['12345678', '87654321', '11223344'],
    'nombre': ['Juan Pérez', 'Ana García', 'Carlos López'],
    'diagnostico': ['Hipertensión', 'Diabetes', 'Asma'],
    'espera_min': [35, 50, 20],
})

# Seudonimización (protección de privacidad)
def seudonimizar(texto):
    return hashlib.sha256(texto.encode()).hexdigest()[:16]

pacientes_anonimo = pacientes.copy()
pacientes_anonimo['dni'] = pacientes_anonimo['dni'].apply(seudonimizar)
pacientes_anonimo['nombre'] = pacientes_anonimo['nombre'].apply(seudonimizar)

print("Datos originales (NO publicables):")
print(pacientes.head())
print("\nDatos seudonimizados (aptos para analytics):")
print(pacientes_anonimo.head())
print(f"\n✅ Tiempo de espera promedio: {pacientes_anonimo['espera_min'].mean():.1f} min")
print("💡 Lineage rastrea que estos datos vienen de registros protegidos")


Datos originales (NO publicables):
        dni        nombre   diagnostico  espera_min
0  12345678    Juan Pérez  Hipertensión          35
1  87654321    Ana García      Diabetes          50
2  11223344  Carlos López          Asma          20

Datos seudonimizados (aptos para analytics):
                dni            nombre   diagnostico  espera_min
0  ef797c8118f02dfb  090fd624fefeeda8  Hipertensión          35
1  e24df920078c3dd4  a18fbef32bea8e51      Diabetes          50
2  4f9f10b304cfe9b2  fc377fa3a02cad5b          Asma          20

✅ Tiempo de espera promedio: 35.0 min
💡 Lineage rastrea que estos datos vienen de registros protegidos


---

## 🧭 Navegación

**← Anterior:** [💳 Finanzas y Banca: riesgo, fraude, rentabilidad](03_finanzas_banca.ipynb)

**Siguiente →:** [⚡ Energía y Recursos Naturales: continuidad, costos y seguridad →](05_energia_recursos_naturales.ipynb)

**📚 Índice de Negocio LATAM:**
- [🎯 Estrategia de Datos en LATAM: de la visión al ROI](01_estrategia_datos_latam.ipynb)
- [🛒 Retail y Consumo Masivo: demanda, surtido y pérdidas](02_retail_consumo_masivo.ipynb)
- [💳 Finanzas y Banca: riesgo, fraude, rentabilidad](03_finanzas_banca.ipynb)
- [🏥 Salud y Farmacéutico: calidad, trazabilidad, acceso](04_salud_farmaceutico.ipynb) ← 🔵 Estás aquí
- [⚡ Energía y Recursos Naturales: continuidad, costos y seguridad](05_energia_recursos_naturales.ipynb)
- [📶 Telecomunicaciones: churn, ARPU y calidad de red](06_telecomunicaciones.ipynb)
- [🏭 Industria y Manufactura: OEE, calidad y throughput](07_industria_manufactura.ipynb)
- [🚚 Logística y Transporte: costo por entrega, SLA, ocupación](08_logistica_transporte.ipynb)
- [🌾 Agro y Alimentos: rendimiento, calidad y trazabilidad](09_agro_alimentos.ipynb)
- [🏛️ Sector Público y Gobierno: transparencia, servicio y eficiencia](10_sector_publico_gobierno.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
